In [2]:
import graphlab

##Read some product review data

In [3]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to joe@joebooth.comand will expire on October 09, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-57130 - Server binary: /Users/joebooth/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1446265881.log
[INFO] GraphLab Server Version: 1.6.1


# explore data

In [4]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


#build world count vector for each review

In [5]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [6]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5, '6': 1,'stink': 1, 'because' ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ..."


#explore Vulli Sophie

In [7]:
graphlab.canvas.set_target('ipynb')

In [8]:
products['name'].show()

In [35]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [36]:
len(giraffe_reviews)

723

In [37]:
giraffe_reviews['rating'].show(view='Categorical')

#build sentiment classifier

In [38]:
products['rating'].show(view='Categorical')

## define positive and a negative sentiment

In [39]:
#ignoral all 3* reviews
products = products[products['rating']!=3]

In [40]:
#positive = 4* or 5* review
products['sentiment'] = products['rating'] >=4

In [41]:
products.head()

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ...",1


#Train sentiment classifier

In [42]:
products['rating'].show(view='Categorical')

In [43]:
train_data, test_data = products.random_split(0.8, seed =0)

In [127]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target = 'sentiment',
                                                     features=['word_count'],
                                                      validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 1.963370     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 3.792363     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 4.468240     | 0.92

#Evaluate the sentiment model 

In [128]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.218079096045 | 0.00552683187734 | 28045 | 5310 |
 | 0.0010000000475  | 0.781920903955 |  0.994473168123  | 28045 | 5310 |
 | 0.00200000009499 | 0.743314500942 |  0.992975574969  | 28045 | 5310 |
 | 0.00300000002608 | 0.721280602637 |  0.992155464432  | 28045 | 5310 |
 | 0.00400000018999 |  0.7065913371  |  0.991406667855  | 28045 | 5310 |
 | 0.00499999988824 | 0.695856873823 |  0.990907470137  | 28045 | 5310 |
 | 0.00600000005215 | 0.686440677966 |  0.99033695846   | 28045 | 5310 |
 | 0.00700000021607 | 0.676271186441 |  0.989873417722  | 28045 | 5310 |
 | 0.00800000037998 | 0.666290018832 |  0.989481190943  | 28045 | 5310 |
 | 0.00899999961257 

In [129]:
sentiment_model.show(view='Evaluation')

#apply model


In [46]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type="probability")

In [47]:
giraffe_reviews.head()


name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1, 'all': 1,'because': 1, 'it': 1, ...",1
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1, 'right': 1,'help': 1, 'just': 1, ...",1
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2, 'all': 1,'would': 1, 'latex.': 1, ...",0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2, 'one!': 1,'all': 1, 'love': 1, ...",1
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3, 'all': 1,'months': 1, 'old': 1, ...",1
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6, 'seven': 1,'already': 1, 'love': 1, ...",1
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4, 'drooling,':1, 'love': 1, ...",1
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3, 'chew': 1,'be': 1, 'is': 1, ...",1
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2, 'seemed': 1,'because': 1, 'about.': ...",1
Vulli Sophie the GiraffeTeether ...,This product is without adoubt the best on the ...,5.0,"{'and': 4, ':)': 1,'just': 2, 'give': 1, ...",1


#sort the review based on the predicted sentiment and export

In [48]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [49]:
giraffe_reviews.head()

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'giggles': 1, 'all': 1,""violet's"": 2, 'bring': ...",1
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'adoring': 1, 'find': 1,'month': 1, 'bright': 1, ...",1
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'all': 2, 'discovered':1, 'existence.': 1, ...",1
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'all': 2, ""don't"": 1,'(literally).so': 1, ...",1
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'cute': 1, 'all': 1,'reviews.': 2, 'just' ...",1
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'just': 2, 'both': 1,'month': 1, 'ears,': 1, ...",1
Vulli Sophie the GiraffeTeether ...,Sophie la giraffe isabsolutely the best toy ...,5.0,"{'and': 5, 'the': 1,'all': 1, 'that': 2, ...",1
Vulli Sophie the GiraffeTeether ...,My 5-mos old son took tothis immediately. The ...,5.0,"{'just': 1, 'shape': 2,'mutt': 1, '""dog': 1, ...",1
Vulli Sophie the GiraffeTeether ...,My nephews and my fourkids all had Sophie in ...,5.0,"{'and': 4, 'chew': 1,'all': 1, 'perfect;': 1, ...",1
Vulli Sophie the GiraffeTeether ...,Never thought I'd see myson French kissing a ...,5.0,"{'giggles': 1, 'all': 1,'out,': 1, 'over': 1, ...",1


In [50]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [30]:
giraffe_reviews[01]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

#show most negative reviews

In [51]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [52]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

# do the assignment

In [53]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']


In [55]:
awesomeCount = graphlab.text_analytics.count_words(products['review'[‘awesome’]])

SyntaxError: invalid syntax (<ipython-input-55-ea145654f47c>, line 1)

In [57]:
aa = giraffe_reviews[-2]['review']

# 1. Use .apply() to build a new feature with the counts for each of the selected_words:

In [70]:
def awesome_count(dict):

    if 'awesome' in dict:

        return (dict['awesome'])

    else:

        return 0

In [71]:
products['word_count'].apply(awesome_count)

dtype: int
Rows: 166752
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... ]

In [72]:
products['awesome']=products['word_count'].apply(awesome_count)

In [90]:
products['awesome'].sum()

333837504

In [76]:
products['awesome']=products['word_count'].apply(awesome_count)

In [84]:
def word_count(dict, word):
    if word in dict:
        return dict[word]
    else:
        return 0

In [88]:
products['word_count'].apply(lambda dict: word_count(dict,'and')).sum()

410789

In [96]:
for word in selected_words:
    #('Currently running against: {}').format(word)
    products[word] = products['word_count'].apply(lambda dict: word_count(dict,word))
    s = '[' + word + ' = ' + format(products[word].sum()) + '] '
    print s

[awesome = 2002] 
[great = 42420] 
[fantastic = 873] 
[amazing = 1305] 
[love = 40277] 
[horrible = 659] 
[bad = 3197] 
[terrible = 673] 
[awful = 345] 
[wow = 131] 
[hate = 1057] 


#2. Create a new sentiment analysis model using only the selected_words as features:

Use the same train/test split as in the IPython Notebook from lecture:

In [98]:
train_data,test_data = products.random_split(.8, seed=0)

Train a logistic regression classifier (use graphlab.logistic_classifier.create) using just the selected_words. Hint: you can use this parameter in the .create() call to specify the features used to be exactly the new columns you just created:

In [101]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target = 'sentiment',
                                                     features=selected_words,
                                                      validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.192279     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.319992     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.441858     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

You will now examine the weights the learned classifier assigned to each of the 11 words in selected_words and gain intuition as to what the ML algorithm did for your data using these features. In GraphLab Create, a learned model, such as the selected_words_model, has a field 'coefficients', which lets you look at the learned coefficients. You can access it by using:

In [103]:
selected_words_model['coefficients']

name,index,class,value
(intercept),None,1,1.36728315229
awesome,None,1,1.05800888878
great,None,1,0.883937894898
fantastic,None,1,0.891303090304
amazing,None,1,0.892802422508
love,None,1,1.39989834302
horrible,None,1,-1.99651800559
bad,None,1,-0.985827369929
terrible,None,1,-2.09049998487
awful,None,1,-1.76469955631


####Using this approach, sort the learned coefficients according to the ‘value’ column using .sort(). Out of the 11 words in selected_words, which one got the most positive weight? Which one got the most negative weight? Do these values make sense for you? Save these results to answer the quiz at the end.

In [106]:
selected_words_model_coefficents = selected_words_model['coefficients']

In [108]:
answer2 = selected_words_model_coefficents.sort('value', ascending=False)
print format(answer2)

+-------------+-------+-------+------------------+
|     name    | index | class |      value       |
+-------------+-------+-------+------------------+
|     love    |  None |   1   |  1.39989834302   |
| (intercept) |  None |   1   |  1.36728315229   |
|   awesome   |  None |   1   |  1.05800888878   |
|   amazing   |  None |   1   |  0.892802422508  |
|  fantastic  |  None |   1   |  0.891303090304  |
|    great    |  None |   1   |  0.883937894898  |
|     wow     |  None |   1   | -0.0541450123333 |
|     bad     |  None |   1   | -0.985827369929  |
|     hate    |  None |   1   |  -1.40916406276  |
|    awful    |  None |   1   |  -1.76469955631  |
+-------------+-------+-------+------------------+
[12 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [204]:
answer2[-1]

{'class': 1, 'index': None, 'name': 'terrible', 'value': -2.090499984872607}

#3. Comparing the accuracy of different sentiment analysis model

In [199]:
selected_words_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+-------------------+----------------+-------+------+
 |    threshold     |        fpr        |      tpr       |   p   |  n   |
 +------------------+-------------------+----------------+-------+------+
 |       0.0        | 0.000189825360668 |      0.0       | 28048 | 5268 |
 | 0.0010000000475  |   0.999810174639  |      1.0       | 28048 | 5268 |
 | 0.00200000009499 |   0.999620349279  |      1.0       | 28048 | 5268 |
 | 0.00300000002608 |   0.999620349279  |      1.0       | 28048 | 5268 |
 | 0.00400000018999 |   0.999430523918  |      1.0       | 28048 | 5268 |
 | 0.00499999988824 |   0.999430523918  |      1.0       | 28048 | 5268 |
 | 0.00600000005215 |   0.999240698557  | 0.999964346834 | 28048 | 5268 |
 | 0.00700000021607 |   0.999240698557  | 0.999964346834 | 28048 | 5268 |
 | 0.00800000037998 |   0.999240698557  | 0.999964346834 | 28048 | 5268 |
 | 0.008

In [200]:
selected_words_model.show(view='Evaluation')

##What is the accuracy of the selected_words_model on the test_data?
#### = accuracy = 0.841

##What was the accuracy of the sentiment_model that we learned using all the word counts in the IPython Notebook above from the lectures? 
#### = accuracy = 0.907

##What is the accuracy majority class classifier on this task? 
#### = 0.161

##How do you compare the different learned models with the baseline approach where we are just predicting the majority class? 
#### = 

In [131]:
giraffe_reviews2 = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [133]:
giraffe_reviews2['predicted_sentiment'] = selected_words_model.predict(giraffe_reviews2, output_type="probability")

In [134]:
giraffe_reviews2.head()

name,review,rating,word_count,sentiment,awesome
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1, 'all': 1,'because': 1, 'it': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1, 'right': 1,'help': 1, 'just': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2, 'all': 1,'would': 1, 'latex.': 1, ...",0,0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2, 'one!': 1,'all': 1, 'love': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3, 'all': 1,'months': 1, 'old': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6, 'seven': 1,'already': 1, 'love': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4, 'drooling,':1, 'love': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3, 'chew': 1,'be': 1, 'is': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2, 'seemed': 1,'because': 1, 'about.': ...",1,0
Vulli Sophie the GiraffeTeether ...,This product is without adoubt the best on the ...,5.0,"{'and': 4, ':)': 1,'just': 2, 'give': 1, ...",1,0


In [135]:
giraffe_reviews2 = giraffe_reviews2.sort('predicted_sentiment', ascending=False)

In [136]:
giraffe_reviews2.head()

name,review,rating,word_count,sentiment,awesome
Vulli Sophie the GiraffeTeether ...,"Great feel, great squeek,great quality, great ...",5.0,"{'rename': 1, 'all': 1,'saving': 1, 'just': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,Love it! Love it! Loveit! The best teether I ...,5.0,"{'several': 1, 'love': 3,':)': 1, 'for': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,Sophie is one of mydaughter's favorite t ...,5.0,"{'and': 1, 'begins': 1,'love': 3, ""daughter's"": ...",1,0
Vulli Sophie the GiraffeTeether ...,We love Sophie at ourhouse... she is a great ...,5.0,"{'and': 4, 'love': 1,'her.': 1, 'is': 5, ' ...",1,0
Vulli Sophie the GiraffeTeether ...,We just got our adorableSophie this week and ...,5.0,"{'and': 4, 'chew': 1,'sees': 1, 'for.': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,Had one for my first twochildren but it had seen ...,5.0,"{'and': 1, 'chew': 1,'love': 2, 'just': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,"Being a childphotographer, I saw lots ...",4.0,"{""don't"": 1, 'being': 2,'son.': 1, 'month': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,This is the first reviewI've written for ...,4.0,"{'dance': 1, 'month': 1,'four': 1, 'bring': 1, ...",1,0
Vulli Sophie the GiraffeTeether ...,My 5mo daughter iscompletely in love with ...,5.0,"{'and': 6, 'chew': 1,'because': 1, 'love': 2, ...",1,0
Vulli Sophie the GiraffeTeether ...,Don't really think thisneeds a review because ...,5.0,"{'and': 3, 'all': 2,'because': 1, 'love': 1, ...",1,0


In [137]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

#4. Interpreting the difference in performance between the models: 

Just like we did for the reviews for the giraffe toy in the IPython Notebook in the lecture video, before we start our analysis you should select all reviews where the product name is ‘Baby Trend Diaper Champ’. Let’s call this table diaper_champ_reviews.

In [185]:
diaper_champ_reviews1 = products[products['name'] == 'Baby Trend Diaper Champ']
diaper_champ_reviews2 = products[products['name'] == 'Baby Trend Diaper Champ']

#####Again, just as in the video, use the sentiment_model to predict the sentiment of each review in diaper_champ_reviews and sort the results according to their ‘predicted_sentiment’.

In [186]:
diaper_champ_reviews1['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews1, output_type="probability")

In [187]:
diaper_champ_reviews1 = diaper_champ_reviews1.sort('predicted_sentiment', ascending=False)

#####What is the ‘predicted_sentiment’ for the most positive review for ‘Baby Trend Diaper Champ’ according to the sentiment_model from the IPython Notebook from lecture? Save this result to answer the quiz at the end.

In [145]:
diaper_champ_reviews1[0]

{'amazing': 0,
 'awesome': 0,
 'awful': 0,
 'bad': 0,
 'fantastic': 0,
 'great': 0,
 'hate': 0,
 'horrible': 0,
 'love': 0,
 'name': 'Baby Trend Diaper Champ',
 'predicted_sentiment': 0.9999999372669541,
 'rating': 5.0,
 'review': 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor e

#####'predicted_sentiment': 0.9999999372669541

#####Now use the selected_words_model you learned using just the selected_words to predict the sentiment most positive review you found above. Hint: if you sorted the diaper_champ_reviews in descending order (from most positive to most negative), this command will be helpful to make the prediction you need:

In [193]:
diaper_champ_reviews2['predicted_sentiment'] = selected_words_model.predict(diaper_champ_reviews2, output_type="probability")

In [194]:
diaper_champ_reviews2 = diaper_champ_reviews2.sort('predicted_sentiment', ascending=False)

In [148]:
diaper_champ_reviews2[0]

{'amazing': 0,
 'awesome': 0,
 'awful': 0,
 'bad': 0,
 'fantastic': 0,
 'great': 1,
 'hate': 0,
 'horrible': 0,
 'love': 3,
 'name': 'Baby Trend Diaper Champ',
 'predicted_sentiment': 0.9984234145936198,
 'rating': 4.0,
 'review': 'I LOVE LOVE LOVE this product! It is SO much easier to use than the Diaper Genie, (you need a PHD in poopy to figure out how to use the darn thing!) and it even takes the same bags as my kitchen trash can, shich is super convenient, and cost efficient as I can buy them in bulk.The only reason for not rating it a 5 star was that I did have one small problem with it. The foam gasket in the barrell which keeps the poopy smell inside the unit ripped somehow, and it got VERY stinky. HOWEVER, I contacted the manufacturer though their website, and received an email back the same day stating that this was unusual, and that replacement gaskets were on their way to me. They arrived inside of a week and after replacing, it works great again! (They even sent me extras s

####'predicted_sentiment': 0.9984234145936198

In [151]:
diaper_champ_reviews2

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,I LOVE LOVE LOVE thisproduct! It is SO much ...,4.0,"{'rating': 1,'contacted': 1, 'over': ...",1,0
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,"{'bags.': 1, ""don't"": 1,'son.': 1, 'of,': 1, ...",1,0
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,"{'instead': 1, 'all': 1,'already': 1, 'love': 3, ...",1,0
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'and': 2, 'bags.': 1,'garbage': 1, 'wastef ...",1,0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'and': 3, 'love': 1,'use.': 1, 'is': 2, ' ...",1,0
Baby Trend Diaper Champ,I've worked with kidsmore than half my life. ...,5.0,"{'and': 4, 'genies': 1,'all': 1, 'because': 1, ...",1,0
Baby Trend Diaper Champ,I love this diaper pail.It keeps the diapers ...,4.0,"{'and': 1, 'old': 1,'extra': 1, 'is': 1, ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3, 'money': 1,'still': 3, 'fine': 1, ...",1,0
Baby Trend Diaper Champ,Love the Diaper Champ. Ihad planned to get the ...,4.0,"{'reviews,': 1, 'infant':1, 'bags.': 1, 'just' ...",1,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'hand.': 1, 'both': 1,'(required': 1, 'befo ...",1,0


##Why is the predicted_sentiment for the most positive review found using the model with all word counts (sentiment_model) much more positive than the one using only the selected_words (selected_words_model)? 
Hint: examine the text of this review, the extracted word counts for all words, and the word counts for each of the selected_words, and you will see what each model used to make its prediction. Save this result to answer the quiz at the end.

In [152]:
diaper_champ_reviews1[0]['review']

'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.'

In [153]:
diaper_champ_reviews2[0]['review']

'I LOVE LOVE LOVE this product! It is SO much easier to use than the Diaper Genie, (you need a PHD in poopy to figure out how to use the darn thing!) and it even takes the same bags as my kitchen trash can, shich is super convenient, and cost efficient as I can buy them in bulk.The only reason for not rating it a 5 star was that I did have one small problem with it. The foam gasket in the barrell which keeps the poopy smell inside the unit ripped somehow, and it got VERY stinky. HOWEVER, I contacted the manufacturer though their website, and received an email back the same day stating that this was unusual, and that replacement gaskets were on their way to me. They arrived inside of a week and after replacing, it works great again! (They even sent me extras should it happen again)I HIGHLY reccomend this diaper pail over ANY competitors, you will not be sorry!'

In [167]:
comp = diaper_champ_reviews1[diaper_champ_reviews1['review'] == 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.']


In [197]:
target = 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.'
for review in diaper_champ_reviews2:
    reviewText = review['review']
    same = reviewText == target
    if same:
        outcome = review
        print review

{'rating': 5.0, 'great': 0, 'love': 0, 'name': 'Baby Trend Diaper Champ', 'sentiment': 1, 'awesome': 0, 'bad': 0, 'fantastic': 0, 'word_count': {'all': 1, 'less': 1, "friend's": 1, '(which': 1, 'absolutly': 2, 'to': 3, 'easy': 2, 'around': 1, 'deffinite': 1, 'luke': 1, 'champ': 1, 'turns': 1, 'bag': 1, 'quick': 1, 'found': 1, 'where': 1, "isn't": 1, 'because': 1, 'house': 1, 'are': 1, 'best': 2, 'really': 3, '"what': 1, 'what': 1, 'for': 2, 'product,': 1, 'seconds': 1, '3': 1, 'integrated': 1, 'dirty': 1, 'we': 2, 'pad)': 1, 'odor': 1, 'use': 1, 'flat.': 1, 'on': 1, 'of': 2, 'chanp': 1, 'turn': 1, 'free,': 1, 'purchase.great': 1, 'reinforced': 1, 'garbage': 1, 'vie': 1, 'into': 2, 'one': 3, 'economical,': 1, 'smelly': 1, 'ties': 1, 'nursery.': 1, 'little': 1, 'from': 1, 'there': 3, 'bjorn,': 1, 'needed': 1, 'was': 2, 'that': 2, 'smell"': 1, 'bulk': 1, 'fabulous.updatei': 1, 'hesitated': 1, 'graco': 1, 'baby': 3, 'champ,': 2, 'champ.': 1, 'than': 1, 'loved': 1, 'this': 1, 'work': 1, 'us

In [188]:
diaper_champ_reviews1[02]['review']

'We researched all of the different types of diaper pails, including the Diaper Genie which some of our friends have, but this one takes the cake. Not only do you not have to buy special bags, the ease of use is phenomenal. Park this thing right next to the changing table, and just drop the diaper in as you go. When you are done, just pull the handle from one side to the other, and presto, you\'re done. It holds around 15-20 diapers (I have changed it probably twice a week), and you just open it, tie the bag and take it out. To make sure there is no smell eminating from it, we put a box of Arm & Hammer in the bottom and use scented garbage bags- and there is NO BAD SMELL AT ALL! I highly recommend that. And you don\'t need top of the line bags, just get some scented off-brand bags (I got 30 for $3), and you should be all set. I would highly recommend this if you are looking for a simple solution. You don\'t need the Genie. My best friend has one, and not only does it stink (literally),

In [198]:
outcome

{'amazing': 0,
 'awesome': 0,
 'awful': 0,
 'bad': 0,
 'fantastic': 0,
 'great': 0,
 'hate': 0,
 'horrible': 0,
 'love': 0,
 'name': 'Baby Trend Diaper Champ',
 'predicted_sentiment': 0.7969408512906712,
 'rating': 5.0,
 'review': 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor e

{'class': 1, 'index': None, 'name': 'terrible', 'value': -2.090499984872607}